# LCA of Bitcoin Mining (attributional model)

Initial set up

In [ ]:
import pandas as pd
import numpy as np
from lci_to_bw2 import *
from brightway2 import *
from matplotlib import pyplot as plt
import time
when = time.strftime("%Y%m%d")

In [ ]:
projects.set_current('BTC_Att_LCA') 

In [ ]:
projects

In [ ]:
#upload APOS attributional ecoinvent v3.5 database (& change directory to find the file on your computer)
fpei34  = "/Users/susanne/Documents/O1a/Databases/Ecoinvent_att_3.5/datasets"
 
if 'ecoinvent 3.5 att' in databases:
     print("Database has already been imported")
else:
     ei34 = SingleOutputEcospold2Importer(fpei34, 'ecoinvent 3.5 att')
     ei34.apply_strategies()
     ei34.statistics()
 
ei34.write_database()

In [ ]:
bw2setup() #do this if its the first time only

In [ ]:
databases

## Data import

Now import the inventory. This has to be done for each model. From here all steps - with the execption of the Monte Carlo simulation - have to be **repeated for the attributional baseline model and all models in the sensitivity anaylsis**.

This means:
1. upload sceanrio the attributional baseline model
2. carry out the LCA and the Monte Carlo simulation for the attributional baseline model
3. upload the sensitivity model +10% electricity
4. carry out the LCA for +10% electricity
5. upload the sensitivity model -10% electricity
6. carry out the LCA for -10% electricity
and so on.

In [ ]:
#change name of file depending on which model is to be run
BTC_att_data = pd.read_csv('Attributional_model.csv', header = 0, sep = ";", encoding = 'utf-8-sig') 

# clean up 
BTC_att_data = BTC_att_data.drop(['Notes'], 1)  # remove the columns not needed
BTC_att_data['Exchange uncertainty type'] = BTC_att_data['Exchange uncertainty type'].fillna(0).astype(int) 

print(BTC_att_data.head())
print(BTC_att_data.tail())  

In [ ]:
# Create a dict that can be written as database
BTC_att_dict = lci_to_bw2(BTC_att_data) # Perfect.
BTC_att_dict

In [ ]:
# Write a bw2 database
databases
if 'BTC_att' in databases: del databases['BTC_att']
BTC_att = Database("BTC_att")
BTC_att.write(BTC_att_dict)
[print(act) for act in BTC_att]

## Static LCA

Now these steps are related to carrying out the LCA. The steps are always conducted on the latest .csv inventory uploaded!

In [ ]:
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod

In [ ]:
acts = []
for activity in Database("BTC_att"):
        acts.append(activity['name'])
acts.sort()
acts

Here the **number behind acts needs to be adapted** to fit the model being analysed (see text below for which number corresponds to which model). When the output says 'Bitcoin Network' (Tera Hashes, None, None) you have chosen the correct value.

In [ ]:
#Change number of acts depending on which model is run
# 26 for attributional model, hashrate, electricity, lifetime
# 2 for Location Global, Hydro, Coal
# 14 for Mining Pools
# 34 for Rauch et al.
# 36 for Bendiksen et al.

myact = Database('BTC_att').get(acts[26])
myact

In [ ]:
functional_unit = {myact: 1} 
lca = LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

The following steps **only** need to be carried out with **the attributional baseline model**. 

In [ ]:
#all analyzed impact categories
mylist = [('IPCC 2013', 'climate change', 'GWP 100a'),
         ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'),
         ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
         ('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
         ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
         ('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
         ('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
         ('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
         ('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'),
         ('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
         ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')]

In [ ]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

def getLCAresults(acts, mymethod):
    
    all_activities = []
    results = []
    for a in acts:
        act = Database('BTC_att').get(a)
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

In [ ]:
results_BTC_att = []
for m in mylist:
    results_all_acts = getLCAresults(acts,m) # total impact per tech
    results_BTC_att.append(results_all_acts)

In [ ]:
methods_names = []
for m in mylist:
    m_name = ' '.join(m)
    methods_names.append(m_name)

In [ ]:
my_output = pd.DataFrame(results_BTC_att, index=methods_names)

In [ ]:
print(my_output['Bitcoin Network'])

In [ ]:
my_output.to_csv('BTC-results_baseline.csv', sep = ';')

## Uncertainty analysis (Monte Carlo simulation)

The Monte Carlo simulation only has to be carried out for the attributional baseline model. This is the case because the Monte Carlo simulation analyses the uncertainty of the background data (ecoinvent database) and the background data is the same for all models.

In [ ]:
mc = MonteCarloLCA({Database('BTC_att').get('Bitcoin Network'):1000000}, mymethod) 
# important to initialize the MC simulation
next(mc)

In [ ]:
# This is the montecarlo simulation
mc = MonteCarloLCA({Database('BTC_att').get('Bitcoin Network'):1000000}, mymethod) 
mc_results = [next(mc) for x in range(1000)]

In [ ]:
from matplotlib.pylab import *
hist(mc_results, density=True)  # From matplotlib package
ylabel("Probability")
print(mean(mc_results))
print(median(mc_results))
np.exp(mean(np.log(mc_results))) 
pd.DataFrame(mc_results).describe()  # Using the pandas package

In [ ]:
df = pd.DataFrame(mc_results)
df.to_csv('MCsimulation_Att_model.csv', sep = ';')